In [ ]:
# This cell is removed with the tag: "remove-input"
# As such, it will not be shown in documentation

# import warnings
# warnings.filterwarnings('ignore')

# Barnase-Barstar

The structure with PDB id [1BRS](https://www.rcsb.org/structure/1BRS) has three barnase-barstar complexes in its unit cell, but none of them has their monomers sequences complete. However, we can find a barnase and a barstar monomer without missing residues coming from different dimers. If we want to prepare a good complex with them, we only need to follow these steps:

    1- Download the molecular system from the protein data bank.
    2- Inspect the molecular system to identify the monomers we want to extract.
    3- Form a new complex with the chosen monomers.
    4- Complete the complex with the missing atoms.
    5- Write the new barnase-barstar complex as a pdb file.

## Download the molecular system

Let's download the 1BRS molecular system with MolSysMT:

In [ ]:
import molsysmt as msm

In [ ]:
molecular_system = msm.convert('1BRS')

## Inspect the molecular system

The basic method {func}`molsysmt.basic.info` it is a good tool to inspect the molecular system at different levels. Let's ask this method for a first overview of the molecular system:

In [ ]:
msm.info(molecular_system)

```{admonition} Note
:class: note
All methods defined in the `molsysmt.basic` module can be invoked also from the main level of the library. As such, `molsysmt.info` is the same method as `molsysmt.basic.info`.
```

The system has water molecules. Let's work only with the proteins.

In [ ]:
molecular_system = msm.extract(molecular_system, selection='molecule_type=="protein"')

In [ ]:
msm.info(molecular_system)

As we said above, {func}`molsysmt.basic.info` can be used to get descriptions of the molecular system at different levels of complexity  looking accordint to its composing elements: atoms, groups, components, chains, molecules or entities.

In [ ]:
msm.info(molecular_system, element='molecule')

In [ ]:
msm.info(molecular_system, element='chain')

This way we get to know that the system is composed by three barnase and three barstar monomers. Those with chain name $B$ and $F$ are longer and compact (they have all their atoms covalently bonded -no missing residues-). Unfortunately, these monomers belong to different dimers as we can see with {func}`molsysmt.basic.view`:

In [ ]:
# This cell is removed with the tag: "remove-input"
# As such, it will not be shown in documentation

nglview_htmlfile = '../../_static/nglview/barnase_barstar_1.html'

In [ ]:
msm.view(molecular_system, viewer='NGLView')

In [ ]:
msm.config._view_from_htmlfiles

In [ ]:
# This cell is removed with the tag: "remove-input"
# As such, it will not be shown in documentation

if False:
    # to write an html the view had to be displayed in a cell before
    msm.thirds.nglview.write_html(view, nglview_htmlfile)

## Form a new complex

To form a new barnase-barstar complex, let's make a structural alignment of the barstar with chain name $F$ over the barstar with chain name $E$. This way, $F$ will be in its bounded position to barnase ($B$). But first let's extract these three monomers:

In [ ]:
barnase = msm.extract(molecular_system, selection="chain_name=='B'")
barstar_E = msm.extract(molecular_system, selection="chain_name=='E'")
barstar_F = msm.extract(molecular_system, selection="chain_name=='F'")

In [ ]:
# This cell is removed with the tag: "remove-input"
# As such, it will not be shown in documentation

nglview_htmlfile = '../../_static/nglview/barnase_barstar_2.html'

In [ ]:
msm.view([barnase, barstar_E, barstar_F], viewer='NGLView')

In [ ]:
# This cell is removed with the tag: "remove-input"
# As such, it will not be shown in documentation

if False:
    # to write an html the view had to be displayed in a cell before
    msm.thirds.nglview.write_html(view, nglview_htmlfile)

We can already work with {func}`molsysmt.structure.align` to overlap the structure of $E$ over $F$:

In [ ]:
barstar_F_over_E = msm.structure.align(barstar_F, selection='atom_name=="CA"',
                                       reference_molecular_system=barstar_E, reference_selection='atom_name=="CA"')

In [ ]:
# This cell is removed with the tag: "remove-input"
# As such, it will not be shown in documentation

nglview_htmlfile = '../../_static/nglview/barnase_barstar_3.html'

In [ ]:
msm.view([barnase, barstar_E, barstar_F_over_E], viewer='NGLView')

In [ ]:
# This cell is removed with the tag: "remove-input"
# As such, it will not be shown in documentation

if False:
    # to write an html the view had to be displayed in a cell before
    msm.thirds.nglview.write_html(view, nglview_htmlfile)

We can finnally produce our new dimer with {func}`molsysmt.basic.merge`:

In [ ]:
barnase_barstar = msm.merge([barnase, barstar_F_over_E])

In [ ]:
# This cell is removed with the tag: "remove-input"
# As such, it will not be shown in documentation

nglview_htmlfile = '../../_static/nglview/barnase_barstar_4.html'

In [ ]:
msm.view(barnase_barstar, viewer='NGLView')

In [ ]:
# This cell is removed with the tag: "remove-input"
# As such, it will not be shown in documentation

if False:
    # to write an html the view had to be displayed in a cell before
    msm.thirds.nglview.write_html(view, nglview_htmlfile)

## Complete the molecular system

Some structures deposited in the Protein Data Bank have atoms with alternate locations, and missing residues or atoms. Let's check if this is the case of our new dimer with the assitance of {func}`molsysmt.build.get_atoms_with_alternate_locations`, {func}`molsysmt.build.get_missing_heavy_atoms`, {func}`molsysmt.build.get_missing_residues`, and {func}`molsysmt.basic.contains`:

In [ ]:
msm.get(barnase_barstar, alternate_locations=True)

In [ ]:
msm.build.get_missing_heavy_atoms(barnase_barstar)

In [ ]:
msm.build.get_missing_residues(barnase_barstar)

In [ ]:
msm.contains(barnase_barstar, selection='hydrogens')

Our complex has no hydrogens -1BRS was solved indeed by x ray crystallography- and some heavy atoms are missing. Let's solve these defects with {func}`molsysmt.build.add_missing_heavy_atoms` and {func}`molsysmt.build.add_missing_hydrogens`:

In [ ]:
barnase_barstar = msm.build.add_missing_heavy_atoms(barnase_barstar)

In [ ]:
barnase_barstar = msm.build.add_missing_hydrogens(barnase_barstar, pH=7.4)

In [ ]:
msm.get(barnase_barstar, element='atom', selection='atom_type=="H"', n_atoms=True)

## Write a pdb file with the new dimer

To conclude this exercise, the new molecular system can be converted to a pdb file:

In [ ]:
msm.convert(barnase_barstar, to_form='barnase_barstar.pdb')

In [ ]:
# This cell is removed with the tag: "remove-input"
# As such, it will not be shown in documentation

from os import remove
remove('barnase_barstar.pdb')